In [2]:
library(wooldridge)
library(stargazer)
library(knitr)

In [8]:
pacman::p_load(wooldridge,stargazer,broom,tidyverse,ggplot2,
               dplyr,tidyr,moderndive,jtools,sjPlot,dotwhisker,FSA,
               car,QuantPsyc,lmtest,sandwich,tseries, 
              dynlm,prais,clubSandwich,orcutt,plm)

Installing package into ‘/home/jupyter/R/x86_64-pc-linux-gnu-library/3.5’
(as ‘lib’ is unspecified)
also installing the dependencies ‘miscTools’, ‘bibtex’, ‘gbRd’, ‘bdsmatrix’, ‘maxLik’, ‘Rdpack’


plm installed



# Chapter 13:

    Example-13-2.R
    Example-13-3-1.R
    Example-13-3-2.R
    Example-13-9.R
    Example-FD.R
    Example-PLM-Calcs.R
    PDataFrame.R





# Example-13-2.R

In [3]:
data(cps78_85, package='wooldridge')

# Detailed OLS results including interaction terms
summary( lm(lwage ~ y85*(educ+female) +exper+ I((exper^2)/100) + union, 
                                                        data=cps78_85) )


Call:
lm(formula = lwage ~ y85 * (educ + female) + exper + I((exper^2)/100) + 
    union, data = cps78_85)

Residuals:
     Min       1Q   Median       3Q      Max 
-2.56098 -0.25828  0.00864  0.26571  2.11669 

Coefficients:
                  Estimate Std. Error t value Pr(>|t|)    
(Intercept)       0.458933   0.093449   4.911 1.05e-06 ***
y85               0.117806   0.123782   0.952   0.3415    
educ              0.074721   0.006676  11.192  < 2e-16 ***
female           -0.316709   0.036621  -8.648  < 2e-16 ***
exper             0.029584   0.003567   8.293 3.27e-16 ***
I((exper^2)/100) -0.039943   0.007754  -5.151 3.08e-07 ***
union             0.202132   0.030294   6.672 4.03e-11 ***
y85:educ          0.018461   0.009354   1.974   0.0487 *  
y85:female        0.085052   0.051309   1.658   0.0977 .  
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1

Residual standard error: 0.4127 on 1075 degrees of freedom
Multiple R-squared:  0.4262,	Adjusted R-squared:  0.4219

# Example-13-3-1.R

In [4]:
data(kielmc, package='wooldridge')

# Separate regressions for 1978 and 1981: report coeeficients only
coef( lm(rprice~nearinc, data=kielmc, subset=(year==1978)) )
coef( lm(rprice~nearinc, data=kielmc, subset=(year==1981)) )

# Joint regression including an interaction term 
library(lmtest)
coeftest( lm(rprice~nearinc*y81, data=kielmc) )

(Intercept)     nearinc 
   82517.23   -18824.37

(Intercept)     nearinc 
  101307.51   -30688.27


t test of coefficients:

            Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  82517.2     2726.9 30.2603 < 2.2e-16 ***
nearinc     -18824.4     4875.3 -3.8612 0.0001368 ***
y81          18790.3     4050.1  4.6395 5.117e-06 ***
nearinc:y81 -11863.9     7456.6 -1.5911 0.1125948    
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


# Example-13-3-2.R

In [6]:
DiD      <- lm(log(rprice)~nearinc*y81                     , data=kielmc)
DiDcontr <- lm(log(rprice)~nearinc*y81+age+I(age^2)+log(intst)+
                            log(land)+log(area)+rooms+baths, data=kielmc)
library(stargazer)
stargazer(DiD,DiDcontr,type="text")


                                  Dependent variable:               
                    ------------------------------------------------
                                      log(rprice)                   
                              (1)                     (2)           
--------------------------------------------------------------------
nearinc                    -0.340***                 0.032          
                            (0.055)                 (0.047)         
                                                                    
y81                        0.193***                 0.162***        
                            (0.045)                 (0.028)         
                                                                    
age                                                -0.008***        
                                                    (0.001)         
                                                                    
I(age2)                          

# Example 13.7: Effect of Drunk Driving Laws on Traffic Fatalities
Wooldridge collected these data from two sources, the 1992 Statistical Abstract of the United States (Tables 1009, 1012) and A Digest of State Alcohol-Highway Safety Related Legislation, 1985 and 1990, published by the U.S. National Highway Traffic Safety Administration. $$\widehat{\Delta{dthrte}} = \beta_0 + \Delta{open} + \Delta{admin}$$

In [2]:
data("traffic1")
?traffic1
DD_model <- lm(cdthrte ~ copen + cadmn, data = traffic1)
stargazer(type = "text",DD_model,  single.row = TRUE, header = FALSE, digits=5)


                        Dependent variable:    
                    ---------------------------
                              cdthrte          
-----------------------------------------------
copen                  -0.41968** (0.20559)    
cadmn                   -0.15060 (0.11682)     
Constant               -0.49679*** (0.05243)   
-----------------------------------------------
Observations                    51             
R2                            0.11867          
Adjusted R2                   0.08194          
Residual Std. Error      0.34350 (df = 48)     
F Statistic           3.23144** (df = 2; 48)   
Note:               *p<0.1; **p<0.05; ***p<0.01


$$\widehat{unemp_t} = \beta_0 + \beta_1unem_{t-1}$$

Estimate the linear model in the usual way and note the use of the subset argument to define data equal to and before the year 1996.


$$\widehat{unemp_t} = \beta_0 + \beta_1unem_{t-1} + \beta_2inf_{t-1}$$



In [6]:
phillips_train <- subset(phillips, year <= 1996)

unem_AR1 <- lm(unem ~ unem_1, data = phillips_train)

In [7]:
unem_inf_VAR1 <- lm(unem ~ unem_1 + inf_1, data = phillips_train)
stargazer(type = "text",unem_AR1, unem_inf_VAR1,  single.row = TRUE, header = FALSE, digits=5)


                                   Dependent variable:               
                    -------------------------------------------------
                                          unem                       
                              (1)                      (2)           
---------------------------------------------------------------------
unem_1                0.73235*** (0.09689)     0.64703*** (0.08381)  
inf_1                                          0.18358*** (0.04118)  
Constant              1.57174*** (0.57712)     1.30380** (0.48969)   
---------------------------------------------------------------------
Observations                   48                       48           
R2                          0.55397                  0.69059         
Adjusted R2                 0.54427                  0.67684         
Residual Std. Error    1.04857 (df = 46)        0.88298 (df = 45)    
F Statistic         57.13184*** (df = 1; 46) 50.21941*** (df = 2; 45)
Note:              

# Example-13-9.R

In [9]:
library(plm);library(lmtest)
data(crime4, package='wooldridge')

crime4.p <- pdata.frame(crime4, index=c("county","year") )
pdim(crime4.p)

# manually calculate first differences of crime rate:
crime4.p$dcrmrte <- diff(crime4.p$crmrte)

# Display selected variables for observations 1-9:
crime4.p[1:9, c("county","year","crmrte","dcrmrte")]


Balanced Panel: n = 90, T = 7, N = 630

,county,year,crmrte,dcrmrte
1-81,1,81,0.0398849,NA
1-82,1,82,0.0383449,-0.0015399978
1-83,1,83,0.0303048,-0.0080401003
1-84,1,84,0.0347259,0.0044211000
1-85,1,85,0.0365730,0.0018470995
1-86,1,86,0.0347524,-0.0018206015
1-87,1,87,0.0356036,0.0008512028
3-81,3,81,0.0163921,NA
3-82,3,82,0.0190651,0.0026730001



t test of coefficients:

              Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)  0.0077134  0.0170579   0.4522 0.6513193    
d83         -0.0998658  0.0238953  -4.1793 3.421e-05 ***
d84         -0.1478033  0.0412794  -3.5806 0.0003744 ***
d85         -0.1524144  0.0584000  -2.6098 0.0093152 ** 
d86         -0.1249001  0.0760042  -1.6433 0.1009087    
d87         -0.0840734  0.0940003  -0.8944 0.3715175    
lprbarr     -0.3274942  0.0299801 -10.9237 < 2.2e-16 ***
lprbconv    -0.2381066  0.0182341 -13.0583 < 2.2e-16 ***
lprbpris    -0.1650463  0.0259690  -6.3555 4.488e-10 ***
lavgsen     -0.0217606  0.0220909  -0.9850 0.3250506    
lpolpc       0.3984264  0.0268820  14.8213 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [10]:

# Estimate FD model:
coeftest( plm(log(crmrte)~d83+d84+d85+d86+d87+lprbarr+lprbconv+ 
                  lprbpris+lavgsen+lpolpc,data=crime4.p, model="fd") )



t test of coefficients:

              Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)  0.0077134  0.0170579   0.4522 0.6513193    
d83         -0.0998658  0.0238953  -4.1793 3.421e-05 ***
d84         -0.1478033  0.0412794  -3.5806 0.0003744 ***
d85         -0.1524144  0.0584000  -2.6098 0.0093152 ** 
d86         -0.1249001  0.0760042  -1.6433 0.1009087    
d87         -0.0840734  0.0940003  -0.8944 0.3715175    
lprbarr     -0.3274942  0.0299801 -10.9237 < 2.2e-16 ***
lprbconv    -0.2381066  0.0182341 -13.0583 < 2.2e-16 ***
lprbpris    -0.1650463  0.0259690  -6.3555 4.488e-10 ***
lavgsen     -0.0217606  0.0220909  -0.9850 0.3250506    
lpolpc       0.3984264  0.0268820  14.8213 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [11]:
library(plm)
data(crime2, package='wooldridge')

# Define panel data frame
crime2.p <- pdata.frame(crime2, index=46 )

# Panel dimensions:
pdim(crime2.p)

# Observation 1-6: new "id" and "time" and some other variables:
crime2.p[1:6,c("id","time","year","pop","crimes","crmrte","unem")]

Balanced Panel: n = 46, T = 2, N = 92

,id,time,year,pop,crimes,crmrte,unem
1-1,1,1,82,229528,17136,74.65756,8.2
1-2,1,2,87,246815,17306,70.11729,3.7
2-1,2,1,82,814054,75654,92.93487,8.1
2-2,2,2,87,933177,83960,89.97221,5.4
3-1,3,1,82,374974,31352,83.61113,9.0
3-2,3,2,87,406297,31364,77.19476,5.9


In [14]:
glimpse(crime2.p)

Observations: 92
Variables: 36
$ pop      <pseries> 229528, 246815, 814054, 933177, 374974, 406297, 176496, …
$ crimes   <pseries> 17136, 17306, 75654, 83960, 31352, 31364, 15698, 16953, …
$ unem     <pseries> 8.2, 3.7, 8.1, 5.4, 9.0, 5.9, 12.6, 5.7, 12.6, 7.4, 13.9…
$ officers <pseries> 326, 321, 1621, 1803, 633, 685, 245, 259, 504, 563, 186,…
$ pcinc    <pseries> 8532, 12155, 7551, 11363, 8343, 11729, 7592, 10802, 7558…
$ west     <pseries> 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,…
$ nrtheast <pseries> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ south    <pseries> 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,…
$ year     <pseries> 82, 87, 82, 87, 82, 87, 82, 87, 82, 87, 82, 87, 82, 87, …
$ area     <pseries> 44.6, 44.6, 375.0, 375.0, 49.8, 49.8, 74.0, 74.0, 97.3, …
$ d87      <pseries> 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0,…
$ popden   <pseries> 5146.368, 5533.969, 2170.811, 2488.472, 7529.599, 8158.5…
$ crmrte   <pseries> 

In [12]:
library(plm);library(lmtest)
data(crime4, package='wooldridge')

# Generate pdata.frame:
crime4.p <- pdata.frame(crime4, index=c("county","year") )

# Estimate FD model:
reg <- ( plm(log(crmrte)~d83+d84+d85+d86+d87+lprbarr+lprbconv+ 
                   lprbpris+lavgsen+lpolpc,data=crime4.p, model="fd") )
# Regression table with standard SE
coeftest(reg)
# Regression table with "clustered" SE (default type HC0):
coeftest(reg, vcovHC)



t test of coefficients:

              Estimate Std. Error  t value  Pr(>|t|)    
(Intercept)  0.0077134  0.0170579   0.4522 0.6513193    
d83         -0.0998658  0.0238953  -4.1793 3.421e-05 ***
d84         -0.1478033  0.0412794  -3.5806 0.0003744 ***
d85         -0.1524144  0.0584000  -2.6098 0.0093152 ** 
d86         -0.1249001  0.0760042  -1.6433 0.1009087    
d87         -0.0840734  0.0940003  -0.8944 0.3715175    
lprbarr     -0.3274942  0.0299801 -10.9237 < 2.2e-16 ***
lprbconv    -0.2381066  0.0182341 -13.0583 < 2.2e-16 ***
lprbpris    -0.1650463  0.0259690  -6.3555 4.488e-10 ***
lavgsen     -0.0217606  0.0220909  -0.9850 0.3250506    
lpolpc       0.3984264  0.0268820  14.8213 < 2.2e-16 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  0.0077134  0.0135800  0.5680 0.5702805    
d83         -0.0998658  0.0219261 -4.5547 6.519e-06 ***
d84         -0.1478033  0.0355659 -4.1558 3.781e-05 ***
d85         -0.1524144  0.0505404 -3.0157 0.0026871 ** 
d86         -0.1249001  0.0623827 -2.0022 0.0457778 *  
d87         -0.0840734  0.0773366 -1.0871 0.2774836    
lprbarr     -0.3274942  0.0555908 -5.8912 6.828e-09 ***
lprbconv    -0.2381066  0.0389969 -6.1058 1.982e-09 ***
lprbpris    -0.1650463  0.0451128 -3.6585 0.0002791 ***
lavgsen     -0.0217606  0.0254368 -0.8555 0.3926740    
lpolpc       0.3984264  0.1014068  3.9290 9.662e-05 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1



t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  0.0077134  0.0137846  0.5596 0.5760131    
d83         -0.0998658  0.0222563 -4.4871 8.865e-06 ***
d84         -0.1478033  0.0361016 -4.0941 4.901e-05 ***
d85         -0.1524144  0.0513017 -2.9709 0.0031038 ** 
d86         -0.1249001  0.0633224 -1.9724 0.0490789 *  
d87         -0.0840734  0.0785015 -1.0710 0.2846678    
lprbarr     -0.3274942  0.0564281 -5.8037 1.118e-08 ***
lprbconv    -0.2381066  0.0395843 -6.0152 3.356e-09 ***
lprbpris    -0.1650463  0.0457923 -3.6042 0.0003427 ***
lavgsen     -0.0217606  0.0258200 -0.8428 0.3997305    
lpolpc       0.3984264  0.1029342  3.8707 0.0001221 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [13]:
# Regression table with "clustered" SE (small-sample correction)
# This is the default version used by Stata and reported by Wooldridge:
coeftest(reg, vcovHC(reg, type="sss"))


t test of coefficients:

              Estimate Std. Error t value  Pr(>|t|)    
(Intercept)  0.0077134  0.0137846  0.5596 0.5760131    
d83         -0.0998658  0.0222563 -4.4871 8.865e-06 ***
d84         -0.1478033  0.0361016 -4.0941 4.901e-05 ***
d85         -0.1524144  0.0513017 -2.9709 0.0031038 ** 
d86         -0.1249001  0.0633224 -1.9724 0.0490789 *  
d87         -0.0840734  0.0785015 -1.0710 0.2846678    
lprbarr     -0.3274942  0.0564281 -5.8037 1.118e-08 ***
lprbconv    -0.2381066  0.0395843 -6.0152 3.356e-09 ***
lprbpris    -0.1650463  0.0457923 -3.6042 0.0003427 ***
lavgsen     -0.0217606  0.0258200 -0.8428 0.3997305    
lpolpc       0.3984264  0.1029342  3.8707 0.0001221 ***
---
Signif. codes:  0 ‘***’ 0.001 ‘**’ 0.01 ‘*’ 0.05 ‘.’ 0.1 ‘ ’ 1


In [ ]:

Stata Textbook Examples
Introductory Econometrics: A Modern Approach by Jeffrey M. Wooldridge (1st & 2nd eds.)
Chapter 13 - Pooling Cross Sections Across Time. Simple Panel Data Methods

    Example 13.1: Woman's Fertility Over Time

use http://fmwww.bc.edu/ec-p/data/wooldridge/fertil1

reg kids educ age agesq black east northcen west farm othrural town smcity y74 y76 y78 y80 y82 y84

      Source |       SS       df       MS              Number of obs =    1129
-------------+------------------------------           F( 17,  1111) =    9.72
       Model |  399.610888    17  23.5065228           Prob > F      =  0.0000
    Residual |  2685.89841  1111  2.41755033           R-squared     =  0.1295
-------------+------------------------------           Adj R-squared =  0.1162
       Total |   3085.5093  1128  2.73538059           Root MSE      =  1.5548

------------------------------------------------------------------------------
        kids |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
        educ |  -.1284268   .0183486    -7.00   0.000    -.1644286    -.092425
         age |   .5321346   .1383863     3.85   0.000     .2606065    .8036626
       agesq |   -.005804   .0015643    -3.71   0.000    -.0088733   -.0027347
       black |   1.075658   .1735356     6.20   0.000     .7351631    1.416152
        east |    .217324   .1327878     1.64   0.102    -.0432192    .4778672
    northcen |    .363114   .1208969     3.00   0.003      .125902    .6003261
        west |   .1976032   .1669134     1.18   0.237    -.1298978    .5251041
        farm |  -.0525575     .14719    -0.36   0.721    -.3413592    .2362443
    othrural |  -.1628537    .175442    -0.93   0.353    -.5070887    .1813814
        town |   .0843532    .124531     0.68   0.498    -.1599893    .3286957
      smcity |   .2118791    .160296     1.32   0.187    -.1026379    .5263961
         y74 |   .2681825    .172716     1.55   0.121    -.0707039    .6070689
         y76 |  -.0973795   .1790456    -0.54   0.587     -.448685    .2539261
         y78 |  -.0686665   .1816837    -0.38   0.706    -.4251483    .2878154
         y80 |  -.0713053   .1827707    -0.39   0.697      -.42992    .2873093
         y82 |  -.5224842   .1724361    -3.03   0.003    -.8608214    -.184147
         y84 |  -.5451661   .1745162    -3.12   0.002    -.8875846   -.2027477
       _cons |  -7.742457   3.051767    -2.54   0.011    -13.73033   -1.754579
------------------------------------------------------------------------------

test y74 y76 y78 y80 y82 y84

 ( 1)  y74 = 0.0
 ( 2)  y76 = 0.0
 ( 3)  y78 = 0.0
 ( 4)  y80 = 0.0
 ( 5)  y82 = 0.0
 ( 6)  y84 = 0.0

       F(  6,  1111) =    5.87
            Prob > F =    0.0000

    Example 13.2: Changes in the Return to Education and the Gender Wage Gap

use http://fmwww.bc.edu/ec-p/data/wooldridge/cps78_85

reg lwage y85 educ y85educ exper expersq union female y85fem

      Source |       SS       df       MS              Number of obs =    1084
-------------+------------------------------           F(  8,  1075) =   99.80
       Model |  135.992074     8  16.9990092           Prob > F      =  0.0000
    Residual |  183.099094  1075  .170324738           R-squared     =  0.4262
-------------+------------------------------           Adj R-squared =  0.4219
       Total |  319.091167  1083   .29463635           Root MSE      =   .4127

------------------------------------------------------------------------------
       lwage |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         y85 |   .1178062   .1237817     0.95   0.341     -.125075    .3606874
        educ |   .0747209   .0066764    11.19   0.000     .0616206    .0878212
     y85educ |   .0184605   .0093542     1.97   0.049      .000106     .036815
       exper |   .0295843   .0035673     8.29   0.000     .0225846     .036584
     expersq |  -.0003994   .0000775    -5.15   0.000    -.0005516   -.0002473
       union |   .2021319   .0302945     6.67   0.000     .1426888    .2615749
      female |  -.3167086   .0366215    -8.65   0.000    -.3885663    -.244851
      y85fem |    .085052    .051309     1.66   0.098    -.0156251     .185729
       _cons |   .4589329   .0934485     4.91   0.000     .2755707     .642295
------------------------------------------------------------------------------

    Example 13.3: Effect of a Garbage Incinerator's Location on Housing Prices

use http://fmwww.bc.edu/ec-p/data/wooldridge/kielmc

reg rprice nearinc if year==1981

      Source |       SS       df       MS              Number of obs =     142
-------------+------------------------------           F(  1,   140) =   27.73
       Model |  2.7059e+10     1  2.7059e+10           Prob > F      =  0.0000
    Residual |  1.3661e+11   140   975815069           R-squared     =  0.1653
-------------+------------------------------           Adj R-squared =  0.1594
       Total |  1.6367e+11   141  1.1608e+09           Root MSE      =   31238

------------------------------------------------------------------------------
      rprice |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
     nearinc |  -30688.27   5827.709    -5.27   0.000    -42209.97   -19166.58
       _cons |   101307.5   3093.027    32.75   0.000     95192.43    107422.6
------------------------------------------------------------------------------

scalar b1=_b[nearinc]

reg rprice nearinc if year==1978

      Source |       SS       df       MS              Number of obs =     179
-------------+------------------------------           F(  1,   177) =   15.74
       Model |  1.3636e+10     1  1.3636e+10           Prob > F      =  0.0001
    Residual |  1.5332e+11   177   866239953           R-squared     =  0.0817
-------------+------------------------------           Adj R-squared =  0.0765
       Total |  1.6696e+11   178   937979126           Root MSE      =   29432

------------------------------------------------------------------------------
      rprice |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
     nearinc |  -18824.37   4744.594    -3.97   0.000    -28187.62   -9461.118
       _cons |   82517.23    2653.79    31.09   0.000     77280.09    87754.37
------------------------------------------------------------------------------

scalar b2=_b[nearinc]

    The difference in two coefficients on nearinc

display b1-b2
-11863.903

reg rprice nearinc y81 y81nrinc

      Source |       SS       df       MS              Number of obs =     321
-------------+------------------------------           F(  3,   317) =   22.25
       Model |  6.1055e+10     3  2.0352e+10           Prob > F      =  0.0000
    Residual |  2.8994e+11   317   914632749           R-squared     =  0.1739
-------------+------------------------------           Adj R-squared =  0.1661
       Total |  3.5099e+11   320  1.0969e+09           Root MSE      =   30243

------------------------------------------------------------------------------
      rprice |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
     nearinc |  -18824.37   4875.322    -3.86   0.000    -28416.45   -9232.293
         y81 |   18790.29   4050.065     4.64   0.000     10821.88    26758.69
    y81nrinc |   -11863.9   7456.646    -1.59   0.113    -26534.67    2806.866
       _cons |   82517.23    2726.91    30.26   0.000      77152.1    87882.36
------------------------------------------------------------------------------

reg rprice nearinc y81 y81nrinc age agesq

      Source |       SS       df       MS              Number of obs =     321
-------------+------------------------------           F(  5,   315) =   44.59
       Model |  1.4547e+11     5  2.9094e+10           Prob > F      =  0.0000
    Residual |  2.0552e+11   315   652459465           R-squared     =  0.4144
-------------+------------------------------           Adj R-squared =  0.4052
       Total |  3.5099e+11   320  1.0969e+09           Root MSE      =   25543

------------------------------------------------------------------------------
      rprice |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
     nearinc |   9397.936   4812.222     1.95   0.052    -70.22392     18866.1
         y81 |   21321.04   3443.631     6.19   0.000     14545.62    28096.47
    y81nrinc |  -21920.27   6359.745    -3.45   0.001    -34433.22   -9407.322
         age |  -1494.424   131.8603   -11.33   0.000    -1753.862   -1234.986
       agesq |   8.691277   .8481268    10.25   0.000     7.022567    10.35999
       _cons |   89116.54   2406.051    37.04   0.000     84382.57     93850.5
------------------------------------------------------------------------------

reg rprice nearinc y81 y81nrinc age agesq intst land area rooms baths

      Source |       SS       df       MS              Number of obs =     321
-------------+------------------------------           F( 10,   310) =   60.19
       Model |  2.3167e+11    10  2.3167e+10           Prob > F      =  0.0000
    Residual |  1.1932e+11   310   384905873           R-squared     =  0.6600
-------------+------------------------------           Adj R-squared =  0.6491
       Total |  3.5099e+11   320  1.0969e+09           Root MSE      =   19619

------------------------------------------------------------------------------
      rprice |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
     nearinc |   3780.334   4453.415     0.85   0.397     -4982.41    12543.08
         y81 |   13928.48   2798.747     4.98   0.000     8421.533    19435.42
    y81nrinc |  -14177.93   4987.267    -2.84   0.005    -23991.11   -4364.759
         age |   -739.451   131.1272    -5.64   0.000    -997.4629   -481.4391
       agesq |    3.45274   .8128214     4.25   0.000     1.853395    5.052084
       intst |  -.5386353   .1963359    -2.74   0.006    -.9249549   -.1523158
        land |   .1414196   .0310776     4.55   0.000     .0802698    .2025693
        area |   18.08621   2.306064     7.84   0.000     13.54869    22.62373
       rooms |   3304.225   1661.248     1.99   0.048     35.47769    6572.973
       baths |   6977.318   2581.321     2.70   0.007     1898.192    12056.44
       _cons |   13807.67   11166.59     1.24   0.217     -8164.23    35779.58
------------------------------------------------------------------------------

reg lprice nearinc y81 y81nrinc

      Source |       SS       df       MS              Number of obs =     321
-------------+------------------------------           F(  3,   317) =   73.15
       Model |  25.1331556     3  8.37771854           Prob > F      =  0.0000
    Residual |  36.3057473   317  .114529171           R-squared     =  0.4091
-------------+------------------------------           Adj R-squared =  0.4035
       Total |  61.4389029   320  .191996572           Root MSE      =  .33842

------------------------------------------------------------------------------
      lprice |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
     nearinc |  -.3399216   .0545554    -6.23   0.000    -.4472581   -.2325851
         y81 |   .4569954   .0453207    10.08   0.000      .367828    .5461628
    y81nrinc |  -.0626505   .0834408    -0.75   0.453    -.2268181    .1015172
       _cons |   11.28542   .0305144   369.84   0.000     11.22539    11.34546
------------------------------------------------------------------------------

    Example 13.4: Effect of Worker Compensation laws on Duration

use http://fmwww.bc.edu/ec-p/data/wooldridge/injury

reg ldurat afchnge highearn afhigh if ky

      Source |       SS       df       MS              Number of obs =    5626
-------------+------------------------------           F(  3,  5622) =   39.54
       Model |  191.071427     3  63.6904757           Prob > F      =  0.0000
    Residual |  9055.93393  5622   1.6108029           R-squared     =  0.0207
-------------+------------------------------           Adj R-squared =  0.0201
       Total |  9247.00536  5625  1.64391206           Root MSE      =  1.2692

------------------------------------------------------------------------------
      ldurat |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
     afchnge |   .0076573   .0447173     0.17   0.864    -.0800058    .0953204
    highearn |   .2564785   .0474464     5.41   0.000     .1634652    .3494918
      afhigh |   .1906012   .0685089     2.78   0.005     .0562973    .3249051
       _cons |   1.125615   .0307368    36.62   0.000     1.065359    1.185871
------------------------------------------------------------------------------

    Example 13.5: Sleeping Versus Working

use http://fmwww.bc.edu/ec-p/data/wooldridge/slp75_81

reg cslpnap ctotwrk ceduc cmarr cyngkid cgdhlth

      Source |       SS       df       MS              Number of obs =     239
-------------+------------------------------           F(  5,   233) =    8.19
       Model |  14674698.2     5  2934939.64           Prob > F      =  0.0000
    Residual |  83482611.7   233  358294.471           R-squared     =  0.1495
-------------+------------------------------           Adj R-squared =  0.1313
       Total |  98157309.9   238  412425.672           Root MSE      =  598.58

------------------------------------------------------------------------------
     cslpnap |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
     ctotwrk |  -.2266694    .036054    -6.29   0.000    -.2977029   -.1556359
       ceduc |  -.0244717   48.75938    -0.00   1.000    -96.09007    96.04113
       cmarr |   104.2139   92.85536     1.12   0.263    -78.72946    287.1574
     cyngkid |    94.6654   87.65252     1.08   0.281    -78.02738    267.3582
     cgdhlth |   87.57785   76.59913     1.14   0.254    -63.33758    238.4933
       _cons |  -92.63404    45.8659    -2.02   0.045    -182.9989   -2.269154
------------------------------------------------------------------------------

test ceduc cmarr cyngkid cgdhlth

 ( 1)  ceduc = 0.0
 ( 2)  cmarr = 0.0
 ( 3)  cyngkid = 0.0
 ( 4)  cgdhlth = 0.0

       F(  4,   233) =    0.86
            Prob > F =    0.4857

    Example 13.6: Distributed Lag of Crime Rate on Clear-up Rate

use http://fmwww.bc.edu/ec-p/data/wooldridge/crime3

reg clcrime cclrprc1 cclrprc2

      Source |       SS       df       MS              Number of obs =      53
-------------+------------------------------           F(  2,    50) =    5.99
       Model |  1.42294706     2  .711473529           Prob > F      =  0.0046
    Residual |  5.93723982    50  .118744796           R-squared     =  0.1933
-------------+------------------------------           Adj R-squared =  0.1611
       Total |  7.36018687    52  .141542055           Root MSE      =  .34459

------------------------------------------------------------------------------
     clcrime |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
    cclrprc1 |  -.0040475   .0047199    -0.86   0.395    -.0135276    .0054326
    cclrprc2 |  -.0131966   .0051946    -2.54   0.014    -.0236302   -.0027629
       _cons |   .0856556   .0637825     1.34   0.185    -.0424553    .2137665
------------------------------------------------------------------------------

    Example 13.7: Effect of Drunk Driving Laws on Traffic Fatalities

use http://fmwww.bc.edu/ec-p/data/wooldridge/traffic1

reg cdthrte copen cadmn

      Source |       SS       df       MS              Number of obs =      51
-------------+------------------------------           F(  2,    48) =    3.23
       Model |  .762579679     2   .38128984           Prob > F      =  0.0482
    Residual |   5.6636945    48  .117993635           R-squared     =  0.1187
-------------+------------------------------           Adj R-squared =  0.0819
       Total |  6.42627418    50  .128525484           Root MSE      =   .3435

------------------------------------------------------------------------------
     cdthrte |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
       copen |  -.4196787   .2055948    -2.04   0.047    -.8330547   -.0063028
       cadmn |  -.1506024   .1168223    -1.29   0.204    -.3854894    .0842846
       _cons |  -.4967872   .0524256    -9.48   0.000    -.6021959   -.3913784
------------------------------------------------------------------------------

    Example 13.8: Effect of Enterprise Zones on Unemployment Claims

use http://fmwww.bc.edu/ec-p/data/wooldridge/ezunem

reg guclms d82 d83 d84 d85 d86 d87 d88 cez

      Source |       SS       df       MS              Number of obs =     176
-------------+------------------------------           F(  8,   167) =   34.50
       Model |  12.8826331     8  1.61032914           Prob > F      =  0.0000
    Residual |  7.79583789   167  .046681664           R-squared     =  0.6230
-------------+------------------------------           Adj R-squared =  0.6049
       Total |   20.678471   175  .118162691           Root MSE      =  .21606

------------------------------------------------------------------------------
      guclms |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         d82 |   .7787595   .0651444    11.95   0.000     .6501469    .9073721
         d83 |  -.0331192   .0651444    -0.51   0.612    -.1617318    .0954934
         d84 |  -.0171382   .0685455    -0.25   0.803    -.1524655     .118189
         d85 |    .323081   .0666774     4.85   0.000     .1914418    .4547202
         d86 |    .292154   .0651444     4.48   0.000     .1635413    .4207666
         d87 |   .0539481   .0651444     0.83   0.409    -.0746645    .1825607
         d88 |  -.0170526   .0651444    -0.26   0.794    -.1456652      .11156
         cez |  -.1818775   .0781862    -2.33   0.021    -.3362382   -.0275169
       _cons |  -.3216319    .046064    -6.98   0.000    -.4125748    -.230689
------------------------------------------------------------------------------

bpagan d82 d83 d84 d85 d86 d87 d88 cez

Breusch-Pagan LM statistic:   6.58428  Chi-sq( 8)  P-value =  .5821

    Example 13.9: Country Crime Rates in North Carolina

use http://fmwww.bc.edu/ec-p/data/wooldridge/crime4

reg clcrmrte d83 d84 d85 d86 d87 clprbarr clprbcon clprbpri clavgsen clpolpc

      Source |       SS       df       MS              Number of obs =     540
-------------+------------------------------           F( 10,   529) =   40.32
       Model |  9.60042816    10  .960042816           Prob > F      =  0.0000
    Residual |  12.5963761   529  .023811675           R-squared     =  0.4325
-------------+------------------------------           Adj R-squared =  0.4218
       Total |  22.1968043   539  .041181455           Root MSE      =  .15431

------------------------------------------------------------------------------
    clcrmrte |      Coef.   Std. Err.      t    P>|t|     [95% Conf. Interval]
-------------+----------------------------------------------------------------
         d83 |  -.0998658   .0238953    -4.18   0.000    -.1468071   -.0529246
         d84 |  -.0479374   .0235021    -2.04   0.042    -.0941063   -.0017686
         d85 |  -.0046111   .0234998    -0.20   0.845    -.0507756    .0415533
         d86 |   .0275143   .0241494     1.14   0.255    -.0199261    .0749548
         d87 |   .0408267   .0244153     1.67   0.095    -.0071361    .0887895
    clprbarr |  -.3274942   .0299801   -10.92   0.000    -.3863889   -.2685994
    clprbcon |  -.2381066   .0182341   -13.06   0.000    -.2739268   -.2022864
    clprbpri |  -.1650462    .025969    -6.36   0.000    -.2160613   -.1140312
    clavgsen |  -.0217607   .0220909    -0.99   0.325    -.0651574    .0216361
     clpolpc |   .3984264    .026882    14.82   0.000     .3456177    .4512351
       _cons |   .0077134   .0170579     0.45   0.651    -.0257961    .0412229
------------------------------------------------------------------------------

whitetst, fitted

White's special test statistic :  118.4921  Chi-sq( 2)  P-value =  1.9e-26 

This page prepared by Oleksandr Talavera (revised 8 Nov 2002)

Send your questions/comments/suggestions to Kit Baum at baum@bc.edu
These pages are maintained by the Faculty Micro Resource Center's GSA Program,
a unit of Boston College Academic Technology Services
